# Testando o monitoramento do gradiente
# Testing the gradient monitoring

In [3]:
import tensorflow as tf
import numpy as np

In [8]:
x = tf.constant(3.0)

with tf.GradientTape(persistent=True) as g:
    g.watch(x)
    y = x*x
    z = y*y

dz_dx = g.gradient(z, x)
dy_dx = g.gradient(y, x)

print(dz_dx)
print(dy_dx)

tf.Tensor(108.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


### O gradiente do x² = 2x, gradiente do x^4 = 4x³, para x = 3 => dy/dx=2.3=6 => dz/dx=3.3³=4.27=108

# Criando a estrutura de multiplicação de matrizes e observabilidade do gradiente

# Monitorando gradiente do Tensor P em relação ao tensor X e W

In [23]:
def product_tensor(X: tf.Tensor, W: tf.Tensor, B: tf.Tensor) -> tuple[tf.Tensor, tf.Tensor]:
    with tf.GradientTape() as g:
        g.watch(X)
        # Compute P as the product of X and W
        P = tf.expand_dims(X, axis=2) * tf.expand_dims(W, axis=0) + tf.expand_dims(B, axis=0)
        P_numpy = P.numpy()
    
        O = tf.math.reduce_prod(P, axis = 1)
        O_numpy = O.numpy()
        
        dO_dX = g.gradient(O, X)
    
    with tf.GradientTape() as g:
        g.watch(W)
        # Compute P as the product of X and W
        P = tf.expand_dims(X, axis=2) * tf.expand_dims(W, axis=0) + tf.expand_dims(B, axis=0)
        P_numpy = P.numpy()
    
        O = tf.math.reduce_prod(P, axis = 1)
        O_numpy = O.numpy()
        
        dO_dW = g.gradient(O, W)

    print(type(dO_dW))
    
    return dO_dX, dO_dW

## Definindo número de variáveis de entrada X, e saidas P

In [24]:
# Define the dimensions
z = 3  # Number of variables in X
n = 5  # Number of inputs in X
k = 2  # Number of outputs in P

## Inicializando variáveis de entrada X

In [25]:
x_array = np.array([[100, 200, 300, 100],
                    [100, 300, 600, 100],
                    [100, 100, 200, 100],
                    [100, 300, 200, 100],
                    [500, 100, 200, 100]])

X = tf.convert_to_tensor(x_array, dtype=tf.float32)
X_numpy = X.numpy()

## Inicializando pesos da rede W

In [26]:
w_array = np.array([[1, 1],
                    [0, 1],
                    [1, 1],
                    [1, 1],])

W = tf.convert_to_tensor(w_array, dtype=tf.float32)
W_numpy = W.numpy()

## Inicializando pesos do viés B

In [27]:
b_array = np.array([[0, 0],
                    [1, 0],
                    [0, 0],
                    [0, 0],])

B = tf.convert_to_tensor(b_array, dtype=tf.float32)
B_numpy = B.numpy()

In [33]:
product_tensor(X, W, B)

<class 'tensorflow.python.framework.ops.EagerTensor'>


(<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
 array([[ 6030000.,  3000000.,  2010000.,  6030000.],
        [18060000.,  6000000.,  3010000., 18060000.],
        [ 2020000.,  2000000.,  1010000.,  2020000.],
        [ 6020000.,  2000000.,  3010000.,  6020000.],
        [ 2020000., 10000000.,  5050000., 10100000.]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[2.3e+07, 4.2e+09],
        [4.2e+09, 4.2e+09],
        [2.3e+07, 4.2e+09],
        [2.3e+07, 4.2e+09]], dtype=float32)>)

In [38]:
def convert_arrays(x_array, w_array, b_array):
    X = tf.convert_to_tensor(x_array, dtype=tf.float32)
    W = tf.convert_to_tensor(w_array, dtype=tf.float32)
    B = tf.convert_to_tensor(b_array, dtype=tf.float32)
    X_numpy = X.numpy()
    W_numpy = W.numpy()
    B_numpy = B.numpy()
    return X, W, B, X_numpy, W_numpy, B_numpy

In [40]:
x_array = np.array([[100, 200, 300, 100],
                    [100, 300, 600, 100],
                    [100, 100, 200, 100],
                    [100, 300, 200, 100],
                    [500, 100, 200, 100]])
w_array = np.array([[1, 1],
                    [1, 1],
                    [1, 1],
                    [1, 1],])
b_array = np.array([[0, 0],
                    [0, 0],
                    [0, 0],
                    [0, 0],])
X, W, B, X_numpy, W_numpy, B_numpy = convert_arrays(x_array, w_array, b_array)

In [41]:
product_tensor(X, W, B)

<class 'tensorflow.python.framework.ops.EagerTensor'>


(<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
 array([[12000000.,  6000000.,  4000000., 12000000.],
        [36000000., 12000000.,  6000000., 36000000.],
        [ 4000000.,  4000000.,  2000000.,  4000000.],
        [12000000.,  4000000.,  6000000., 12000000.],
        [ 4000000., 20000000., 10000000., 20000000.]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[4.2e+09, 4.2e+09],
        [4.2e+09, 4.2e+09],
        [4.2e+09, 4.2e+09],
        [4.2e+09, 4.2e+09]], dtype=float32)>)

In [29]:
print("X Variable:")
print(X_numpy)
print("W Variable:")
print(W_numpy)
print("B Variable:")
print(B_numpy)
print("d/dx")
print(dO_dX)
print("d/dw")
print(dO_dW)

X Variable:
[[100. 200. 300. 100.]
 [100. 300. 600. 100.]
 [100. 100. 200. 100.]
 [100. 300. 200. 100.]
 [500. 100. 200. 100.]]
W Variable:
[[1. 1.]
 [0. 1.]
 [1. 1.]
 [1. 1.]]
B Variable:
[[0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]]
d/dx
tf.Tensor(
[[ 6030000.  3000000.  2010000.  6030000.]
 [18060000.  6000000.  3010000. 18060000.]
 [ 2020000.  2000000.  1010000.  2020000.]
 [ 6020000.  2000000.  3010000.  6020000.]
 [ 2020000. 10000000.  5050000. 10100000.]], shape=(5, 4), dtype=float32)
d/dw
tf.Tensor(
[[2.3e+07 4.2e+09]
 [4.2e+09 4.2e+09]
 [2.3e+07 4.2e+09]
 [2.3e+07 4.2e+09]], shape=(4, 2), dtype=float32)


In [ ]:
w_array = np.array([[1, 1],
                    [-0.01, 1],
                    [1, 1],
                    [1, 1],])

W = tf.convert_to_tensor(w_array, dtype=tf.float32)
W_numpy = W.numpy()

b_array = np.array([[0, 0],
                    [0.99, 0],
                    [0, 0],
                    [0, 0],])

B = tf.convert_to_tensor(b_array, dtype=tf.float32)
B_numpy = B.numpy()